In [ ]:
# Adventures on Machine Learning (1)
# Handling with text and forecasting
# by Paulo Francisco da Silva Junior

In [ ]:
#Importações
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import numpy as np

In [ ]:
#Montando base de dados na mão
#Para importar, verificar comando load_files do sklearn
treinamento = {'data':[
     'Futebol é interessante',
     'adrenalina pura na luta',
     'parece box com lutadores loucos para vencer',
     'mas prefiro hacking',
     'viagen neural',
     'jogando bola'
     ],
     'target_name':[
                     "futebol",
                     "luta",
                     "luta",
                     "computação",
                     "computação",
                     "futebol"
     ],
     "target":[0,1,1,2,2,0]}
validacao = {"data":[
              'Hacking é computação',
              'hackear o mundo faz parte de todo ciêntista',
              'volei é diferente do futebol',
              'box é uma luta'],
     'target_name':[
                     "computação",
                     "computação",
                     "futebol",
                     "luta"],
     "target":[2,2,0,1]}

In [ ]:
#Criando um dicionario da base (irrelevante)
cache = dict(train=treinamento,test=validacao)

In [ ]:
cache["train"]

{'data': ['Futebol é interessante',
  'adrenalina pura na luta',
  'parece box com lutadores loucos para vencer',
  'mas prefiro hacking',
  'viagen neural',
  'jogando bola'],
 'target': [0, 1, 1, 2, 2, 0],
 'target_name': ['futebol',
  'luta',
  'luta',
  'computação',
  'computação',
  'futebol']}

In [ ]:
#Como utilizar esse dicionário por chaves
cache['train']['data'][0]
#Anexando parte do dicionario ao uma variável separada
treinamentoData=cache["train"]
validacaoData=cache["test"]

In [ ]:
treinamentoData["target"] #Verificando os "alvos", respostas corretas

[0, 1, 1, 2, 2, 0]

In [ ]:
vetorizarPalavras=CountVectorizer(ngram_range=(1,1)) #criando instância do obj

In [ ]:
cxt=vetorizarPalavras.fit_transform(treinamentoData["data"]) #Vetorizando palavras, colocando todas palavras (sem repetições caso ngram=(1,1)) num vetor. 

In [ ]:
vetorizarPalavras.get_feature_names()

['adrenalina',
 'bola',
 'box',
 'com',
 'futebol',
 'hacking',
 'interessante',
 'jogando',
 'loucos',
 'luta',
 'lutadores',
 'mas',
 'na',
 'neural',
 'para',
 'parece',
 'prefiro',
 'pura',
 'vencer',
 'viagen']

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [ ]:
tfidtransformation=TfidfTransformer() #criando instância do obj

In [ ]:
cxtTFID= tfidtransformation.fit_transform(cxt) #Transforma uma matriz de contagem em uma representação de frequência de termo

In [ ]:
cxt = vetorizarPalavras.fit_transform(treinamentoData["data"]) #Vetorizando as palavras novamente

In [ ]:
 #criação do modelo
 #Pipeline -> construtor
 #Parametros através de [()] -> tuplas dentro de lista
 #Tuplas seguem o seguinte formato -> ("nome arbitrário da etapa",função_parametro_modelo(args))
 #Especificações desse modelo -> Palavras vetorizadas (1,1), Frequência de palavras ajustadas (idf=false), não saber probabilidades (MultinomialNB(fit_prior=False)) 
modelo = Pipeline([("vetorWord",CountVectorizer(ngram_range=(1,1),max_df=(0.75))),("tfd",TfidfTransformer(use_idf=False)),("ctf", MultinomialNB(fit_prior=False))])

In [ ]:
#Aprendizado da maquina
#OBS: Dados de treinamento
modelo.fit(treinamentoData["data"],treinamentoData["target"])

Pipeline(memory=None,
         steps=[('vetorWord',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.75,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfd',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=False)),
                ('ctf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False))],
         verbose=False)

In [ ]:
#Validando predição dos dados
prever = modelo.predict(validacaoData["data"])

In [ ]:
prever #Obtendo resposta da previsão do modelo

array([2, 0, 0, 1])

In [ ]:
validacaoData["target"] #Resgatando as respostas corretas da previsão

[2, 2, 0, 1]

In [ ]:
prever == validacaoData["target"] #Comparando os resultados da previsão com a base de validação

array([ True, False,  True,  True])

In [ ]:
np.mean(prever==validacaoData["target"]) #Média de probabilidade da comparação

0.75

In [ ]:
#GridSearchCv -> Pesquisa exaustiva sobre valores de parâmetro especificados para um estimador.
#Modelo deve ser passado, assim como possíveis parametros, n_job->nº processadores
#cv -> numero de "dobras" de aprendizado. cn < nº de grupos ou nº 
#parametros como -> nomeIntânciaModelo__parametromodelo
parametros ={"vetorWord__ngram_range":[(1,1),(1,2),(2,2),(2,3),(3,3)],"vetorWord__max_df":(0.25,0.5,0.75,0.9),'tfd__use_idf': (True, False),"ctf__fit_prior":(True,False)}
gr=GridSearchCV(modelo, parametros,n_jobs=2,cv=2)

In [ ]:
#Realizando previsão dos melhores parametros
modelo_novo = gr.fit(treinamentoData["data"], treinamentoData["target"])

In [ ]:
#Melhor previsão do modelo
#Note que são varias interações, varios parametros sendo testados
gr.best_score_

0.3333333333333333

In [ ]:
#Melhores parametros do modelo
gr.best_params_

{'ctf__fit_prior': True,
 'tfd__use_idf': True,
 'vetorWord__max_df': 0.5,
 'vetorWord__ngram_range': (1, 1)}

In [ ]:
#Prevendo através do novo modelo
previstogr = gr.predict(validacaoData["data"])
previstogr

array([2, 0, 0, 1])

In [ ]:
#Comparando modelos (bool)
validacaoData["target"]==previstogr

array([ True, False,  True,  True])

In [ ]:
#Taxa de acerto (média das probabilidades)
np.mean(validacaoData['target']==previstogr)

0.75

In [ ]:
nunes_teste = {"data":[
                       "magrão é um ótimo goleiro",
                       "eu gosto de derrubar e bater",
                       "apostar corrida de cavalo da dinheiro",
                       "eu sou apostador compulsivo",
                       "eu amo a sua vida",
                       "como posso ser seu amigo cara?"
  ],"target_name":["futebol","luta", "sorte","sorte","homosexual","homosexual"], "target":[0,1,2,2,3,3]
}
nunes_treinamento = {"data":
                    [""]
                     }